# Standard Pooling Problem

## Objective

One of the new features of Gurobi 9.0 is the addition of a bilinear solver, which enables finding the optimal solution of non-convex quadratic programming  problems (i.e. QPs, QCQPs, MIQPs and MIQCQPs). This notebook will show you how to use this feature by tackling an instance of the standard pooling problem, one of the most well-known bilinear programs. Moreover, we will present two alternative formulations, casted as Quadratically Constrained Quadratic Problems, to contrast their performance when calling the optimizer.

---
## Motivation

The pooling problem is a challenging problem in the petrochemical refining, wastewater treatment and mining industries. This problem can be regarded as a generalization of the minimum-cost flow problem and the blending problem. It is indeed important because of the significant savings it can generate, so it comes at no surprise that it has been studied extensively since Haverly pointed out the non-linear structure of this problem in 1978 [5].

---
## Problem Description

The Minimum-Cost Flow Problem (MCFP) seeks to find the cheapest way of sending a certain amount of flow from a set of source nodes to a set of target nodes, possibily via transshipment nodes⁠, in a directed capacitated network. The Blending Problem is a type of MCFP with only source and target nodes, where raw materials with different attribute qualities are blended together to create end products in such a way that their attribute qualities are within tolerances.

The Pooling Problem combines features of both problems, as flow streams from different sources are mixed at intermediate pools and blended again at the target nodes. The non-linearity is in fact the direct result of considering pools, as the quality of a given attribute at a pool —defined as the weighted average of the qualities of the incoming streams— is an unknown quantity and thus needs to be captured by a decision variable. We refer to this problem as the Standard Pooling Problem when the network can be represented by a tripartite graph, i.e. three disjoint sets of nodes such that no nodes within the same set are adjacent. In a nutshell, it can be stated as follows: Given a list of source nodes with raw materials containing known attribute qualities, what is the cheapest way of mixing these materials at intermediate pools so as to meet the demand and tolerances at multiple target nodes? (Gupte et al., 2017) [4]. Several different formulations for the Standard Pooling Problem and its extensions exist in the literature, which can be classified into two main categories: one that consists of flow and quality variables, and the other that uses flow proportions instead of quality variables. Both categories will be considered in this notebook.

---
## Problem Instance

As an illustrative example, we will solve the second pooling problem posed by Rehfeldt and Tisljar in 1997 and cited by Audet et al. in 2004:

![Graph](rehfeldt_tisljar2_graph.png)

![Table](rehfeldt_tisljar2_table.png)

To that end, let's declare the required data structures to represent this problem instance:

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from gurobipy import *

attrs = {'den', 'bnz', 'roz', 'moz'}

sources, cost, supply, content = multidict({
    "s1": [49.2, 6097.56, {'den': 0.82, 'bnz':3, 'roz':99.2,'moz':90.5}],
    "s2": [62.0, 16129, {'den': 0.62, 'bnz':0, 'roz':87.9,'moz':83.5}],
    "s3": [300.0, 500, {'den': 0.75, 'bnz':0, 'roz':114,'moz':98.7}]
})

targets, price, demand, min_tol, max_tol = multidict({
    "t1": [190, 500, {'den': 0.74, 'roz':95,'moz':85}, {'den': 0.79}],
    "t2": [230, 500, {'den': 0.74, 'roz':96,'moz':88}, {'den': 0.79, 'bnz':0.9}],
    "t3": [150, 500, {'den': 0.74, 'roz':91}, {'den': 0.79}]
})

pools, cap = multidict({
    "p1": 1250,
    "p2": 1750
})

# The function `product` deploys the Cartesian product of elements in sets A and B
s2p = set(product(sources, pools))
p2t = set(product(pools, targets))
s2t = {("s1", "t2"),
       ("s2", "t1"),
       ("s2", "t3"),
       ("s3", "t1")}

---
## Solution Approach

Mathematical programming is a declarative approach where the modeler formulates a mathematical optimization model that captures the key aspects of a complex decision problem. The Gurobi optimizer solves such models using state-of-the-art mathematics and computer science.

A mathematical optimization model has five components, namely:

- Sets and indices.
- Parameters.
- Decision variables.
- Objective function(s).
- Constraints.

A quadratic constraint that involves only products of disjoint pairs of variables is often called a bilinear constraint, and a model that contains bilinear constraints is often called a Bilinear Program. Bilinear constraints are a special case of non-convex quadratic constraints. This type of problems are typically solved using spatial Branch and Bound (sB&B). This algorithm explores the entire search space, so it provides a globally valid lower bound on the optimal objective value and —given enough time— it will find a globally optimal solution (subject to tolerances). The interested readers is referred to [references](#references) [3], [6] and [7].

We now present two alternative Bilinear Programs for the Standard Pooling Problem:

### P-formulation (Concentration)

#### Sets and Indices

$G=(V,E)$: Directed graph.

$i,j \in V$: Set of nodes.

$(i,j) \in E \subset  V \times V$: Set of edges.

$N(i)^+ \{j \in V \mid (i,j) \in E \}$: Set of successor nodes receiving outflow from node $i$.

$N(j)^- \{i \in V \mid (i,j) \in E \}$: Set of predecessor nodes sending inflow to node $i$.

$k \in \text{Attrs}$: Set of attributes.

$s \in \text{Sources} \subset V$: Set of source nodes, i.e. $N(s)^-= \emptyset$.

$t \in \text{Targets} \subset V$: Set of target nodes, i.e. $N(t)^+= \emptyset$.

$p \in \text{Pools} = V \setminus (\text{Sources} \cup \text{Targets})$: Set of pools.

#### Parameters

$\text{Cost}_s \in \mathbb{R}^+$: Cost of acquiring one unit of raw material at source node $s$.

$\text{Supply}_s \in \mathbb{R}^+$: Maximum number of units of raw material available at source node $s$.

$\text{Content}_{s,k} \in \mathbb{R}^+$: Content of attribute $k$ in raw material at source node $s$.

$\text{Price}_t \in \mathbb{R}^+$: Price for selling one unit of final blend at target node $t$.

$\text{Demand}_t \in \mathbb{R}^+$: Minimum number of units required of final blend at target node $t$.

$\text{Min_tol}_{t,k} \in \mathbb{R}^+$: Minimum tolerance for attribute $k$ in final blend at target node $t$.

$\text{Max_tol}_{t,k} \in \mathbb{R}^+$: Maximum tolerance for attribute $k$ in final blend at target node $t$.

$\text{Cap}_p \in \mathbb{R}^+$: Maximum Capacity to store intermediate blend at pool $p$.

$\text{UB}_{i,j}\in \mathbb{R}^+$: Maximum flow from node $i$ to node $j$.

#### Decision Variables

$\text{flow}_{i,j} \in [0, \text{UB}_{i,j}]$: Flow from node $i$ to node $j$.

$\text{quality}_{p,k} \in \mathbb{R}^+$: Concentration of attribute $k$ at pool $p$.

#### Objective Function

- **Profit**: Maximize total profits.

\begin{equation}
\text{Max} \quad Z = \sum_{t \in \text{Targets}}{\sum_{i \in N(t)^-}{\text{Price}_t \cdot \text{flow}_{i,t}}} - \sum_{s \in \text{Sources}}{\sum_{j \in N(s)^+}{\text{Cost}_s \cdot \text{flow}_{s,j}}}
\tag{0}
\end{equation}

#### Constraints

- **Flow conservation**: Total inflow of pool $p$ must be equal to its toal outflow (nothing is stored in them).

\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} - \sum_{s \in N(p)^-}{\text{flow}_{s,p}} = 0 \quad \forall p \in \text{Pools}
\tag{1}
\end{equation}

- **Source capacity**: Total outflow of source $s$ cannot exceed its capacity.

\begin{equation}
\sum_{j \in N(s)^+}{\text{flow}_{s,j}} \leq \text{Supply}_s \quad \forall s \in \text{Sources}
\tag{2}
\end{equation}

- **Pool capacity**: Total outflow of pool $p$ cannot exceed its capacity.

\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{Cap}_p \quad \forall p \in \text{Pools}
\tag{3}
\end{equation}

- **Target demand**: Total inflow of target $t$ must at least meet its minimum demand.

\begin{equation}
\sum_{i \in N(t)^-}{\text{flow}_{i,t}} \geq \text{Demand}_t \quad \forall t \in \text{Targets}
\tag{4}
\end{equation}

- **Pool concentration**: Concentration of attribute $k$ at pool $p$ is expressed as the weighted average (linear blending) of the concentrations associated to the incoming flows (notice the bilinear terms on the right-hand side).

\begin{equation}
\sum_{s \in N(p)^-}{\text{Content}_{s,k} \cdot \text{flow}_{s,p}} = \text{quality}_{p,k} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}} \quad \forall (p,k) \in \text{Pools} \times \text{Attrs}
\tag{5}
\end{equation}

- **Target tolerances**: Concentration of attribute $k$ at target $t$ is also the result of linear blending, and must be within tolerances (notice the bilinear terms on the second expression of the left-hand side).

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}}+ \sum_{p \in N(t)^- \cap \text{Pools}}{\text{quality}_{p,k} \cdot \text{flow}_{p,t}} \geq \text{Min_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \quad \forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{6.1}
\end{equation}

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}}+ \sum_{p \in N(t)^- \cap \text{Pools}}{\text{quality}_{p,k} \cdot \text{flow}_{p,t}} \leq \text{Max_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \quad \forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{6.2}
\end{equation}

The number of bilinear terms in this formulation is proportional to the number of attributes. An alternative formulation relies on decision variables that represent fractions of flow instead of concentrations, so that the bilinear terms are no longer associated to the number of attributes. Two types of decision variables may be used:

- fraction of total inflow at pool $p$, coming from source $s$.
- fraction of total outflow at pool $p$, going to terminal $t$.

A model based on the first option is now presented:

### Q-formulation (Proportion)

#### Decision Variables

$\text{flow}_{i,j} \in [0, \text{UB}_{i,j}]$: Flow from node $i$ to node $j$.

$\text{prop}_{s,p} \in \mathbb{R}^+$: fraction of total inflow at pool $p$, coming from source $s$.

#### Objective Function

- **Profit**: Maximize total profits (notice the bilinear terms on the second expression).

\begin{equation}
\text{Max} \quad Z = \sum_{t \in \text{Targets}}{\sum_{i \in N(t)^-}{\text{Price}_t \cdot \text{flow}_{i,t}}} - \sum_{s \in \text{Sources}}{\text{cost}_s \cdot \left( \sum_{t \in N(s)^+ \cap \text{Targets}}{\text{flow}_{s,t}} + \sum_{p \in N(s)^+ \cap \text{Pools}}{\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}}} \right) }
\tag{0}
\end{equation}

#### Constraints

- **Source capacity**: Total outflow of source $s$ cannot exceed its capacity (notice the bilinear terms on the first expression of the left-hand side).

\begin{equation}
\sum_{p \in N(s)^+ \cap \text{Pools}}{\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}}} + \sum_{t \in N(s)^+ \cap \text{Targets}}{\text{flow}_{s,t}} \leq \text{Supply}_s \quad \forall s \in \text{Sources}
\tag{1}
\end{equation}

- **Pool capacity**: Total outflow of pool $p$ cannot exceed its capacity.

\begin{equation}
\sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{Cap}_p \quad \forall p \in \text{Pools}
\tag{2}
\end{equation}

- **Target demand**: Total inflow of target $t$ must at least meet its minimum demand.

\begin{equation}
\sum_{i \in N(t)^-}{\text{flow}_{i,t}} \geq \text{Demand}_t \quad \forall t \in \text{Targets}
\tag{3}
\end{equation}

- **Pool inflow**: The sum of the contributions of all incoming sources to pool $p$ must equal one.

\begin{equation}
\sum_{s \in N(p)^-}{\text{prop}_{s,p}} = 1 \quad \forall p \in \text{Pools}
\tag{4}
\end{equation}

- **Target tolerances**: Concentration of attribute $k$ at target $t$ is also the result of linear blending, and must be within tolerances (notice the bilinear terms on the second expression of the left-hand side).

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}} + \sum_{p \in N(t)^- \cap \text{Pools}}{\text{flow}_{p,t} \cdot \sum_{s \in N(p)^-}{\text{content}_{s,k} \cdot \text{prop}_{s,p}}}
\geq \text{Min_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \\
\forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{5.1}
\end{equation}

\begin{equation}
\sum_{s \in N(t)^- \cap \text{Sources}}{\text{Content}_{s,k} \cdot \text{flow}_{s,t}} + \sum_{p \in N(t)^- \cap \text{Pools}}{\text{flow}_{p,t} \cdot \sum_{s \in N(p)^-}{\text{content}_{s,k} \cdot \text{prop}_{s,p}}}
\leq \text{Max_tol}_{t,k} \cdot \sum_{i \in N(t)^-}{\text{flow}_{i,t}} \\
\forall (t,k) \in \text{Targets} \times \text{Attrs}
\tag{5.2}
\end{equation}

One drawback is that if some of the source-to-pool edges have flow capacity, we need to define additional constraints instead of just specifying upper bounds on the associated decision variables. Such constraints can be defined with bilinear terms as follows:

- **Flow limit**: Flow from source $s$ to pool $p$ cannot exceed the installed capacity (notice the bilinear terms on the left-hand side).

\begin{equation}
\text{prop}_{s,p} \cdot \sum_{t \in N(p)^+}{\text{flow}_{p,t}} \leq \text{UB}_{s,p} \quad \forall (i,j) \in E \cap \left( \text{Sources} \times \text{Pools} \right) \mid \text{UB}_{i,j} < \infty
\tag{6}
\end{equation}

---
## Python Implementation

Solving Bilinear Programs with Gurobi is as easy as configuring the global parameter `nonConvex`. When setting this parameter to a value of 2, non-convex quadratic problems are solved by means of translating them into bilinear form and applying sB&B. We will first deploy the P-formulation model, and then compare it with the Q-formulation model:

### P-formulation (Concentration)

In [2]:
p_pooling = Model("Pooling")

# Set global parameters
p_pooling.params.nonConvex = 2
p_pooling.params.timelimit = 5*60 # time limit of 5 minutes

# Declare decision variables

# flow
ik = p_pooling.addVars(s2t, name="Source2Target")
ij = p_pooling.addVars(s2p, name="Source2Pool")
jk = p_pooling.addVars(p2t, name="Pool2Target")
ik["s1","t2"].ub = 750
ik["s3","t1"].ub = 750
# quality
prop = p_pooling.addVars(pools, attrs, name="Proportion")

# Deploy constraint sets

# 1. Flow conservation
p_pooling.addConstrs((ij.sum('*',j) == jk.sum(j,'*') for j in pools), name="Flow_conservation")
# 2. Source capacity
p_pooling.addConstrs((ij.sum(i,'*') + ik.sum(i,'*') <= supply[i] for i in sources), name="Source_capacity")
# 3. Pool capacity
p_pooling.addConstrs((jk.sum(j,'*') <= cap[j] for j in pools), name="Pool_capacity")
# 4. Target demand
p_pooling.addConstrs((ik.sum('*',k) + jk.sum('*',k) >= demand[k] for k in targets), name="Target_demand")
# 5. Pool concentration
p_pooling.addConstrs((quicksum(content[i][attr]*ij[i,j] for i in sources if (i,j) in s2p) == prop[j,attr]*jk.sum(j,'*') for j in pools for attr in attrs), name="Pool_concentration")
# 6.1 Target (min) tolerances
p_pooling.addConstrs((quicksum(content[i][attr]*ik[i,k] for i in sources if (i,k) in s2t)+quicksum(prop[j,attr]*jk[j,k] for j in pools if (j,k) in p2t)>= min_tol[k][attr]*(ik.sum('*',k)+jk.sum('*',k)) for k in targets for attr in min_tol[k].keys()), name="Target_min_tolerances")
# 6.2 Target (max) tolerances
p_pooling.addConstrs((quicksum(content[i][attr]*ik[i,k] for i in sources if (i,k) in s2t)+quicksum(prop[j,attr]*jk[j,k] for j in pools if (j,k) in p2t)<= max_tol[k][attr]*(ik.sum('*',k)+jk.sum('*',k)) for k in targets for attr in max_tol[k].keys()), name="Target_max_tolerances")

# Deploy Objective Function

# 0. Total profit
obj = quicksum(price[k]*(ik.sum('*',k)+jk.sum('*',k)) for k in targets) - quicksum(cost[i]*(ij.sum(i,'*')+ik.sum(i,'*')) for i in sources)
p_pooling.setObjective(obj, GRB.MAXIMIZE)

# Find the optimal solution
p_pooling.optimize()

Gurobi 9.0 beta0 - expires 2019-11-30
Using license file /Users/orojuan/gurobi.lic
Set parameter TokenServer to value Juans-MacBook-Pro-3.local
Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Changed value of parameter timelimit to 300.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0beta1 (mac64)
Optimize a model with 10 rows, 24 columns and 38 nonzeros
Model fingerprint: 0x07535d75
Model has 20 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [1e+00, 1e+00]
  QLMatrix range   [1e-02, 1e+02]
  Objective range  [5e+01, 3e+02]
  Bounds range     [8e+02, 8e+02]
  RHS range        [5e+02, 2e+04]

Continuous model is non-convex -- solving as a MIP.

Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 125 rows, 49 columns, 311 nonzeros
Presolved model has 24 bilinear constraint(s)
Variable types: 49 continuous, 0 integer (0 binary)

Root relaxati

The P-formulation for this instance has:

- 24 decision variables.
- 10 linear constraints.
- 20 bilinear constraints.
- a linear objective function.

As can be seen, we still observe a gap of 70.10% after reaching the time limit of five minutes (at this point, the incumbent solution induces a total profit of 415,473.30 USD). In fact, even after 20 minutes the solver does not make much progress in closing the gap.

### Q-Formulation (proportion)

Let's now see how the q-formulation model performs:

In [3]:
q_pooling = Model("Pooling")

# Set global parameters
q_pooling.params.nonConvex = 2
p_pooling.params.timelimit = 5*60

# Declare decision variables

# flow
ik = q_pooling.addVars(s2t, name="Source2Target")
jk = q_pooling.addVars(p2t, name="Pool2Target")
ik["s1","t2"].ub = 750
ik["s3","t1"].ub = 750
# quality
p_ij = q_pooling.addVars(s2p, ub=1.0, name="Prop_Source2Pool")

# Deploy constraint sets

# 1. Source capacity
q_pooling.addConstrs((quicksum(p_ij[i,j]*jk.sum(j,'*') for j in pools if (i,j) in s2p) + ik.sum(i,'*') <= supply[i] for i in sources), name="Source_capacity")
# 2. Pool capacity
q_pooling.addConstrs((jk.sum(j,'*') <= cap[j] for j in pools), name="Pool_capacity")
# 3. Target demand
q_pooling.addConstrs((ik.sum('*',k) + jk.sum('*',k) >= demand[k] for k in targets), name="Target_demand")
# 4. Pool inflow
q_pooling.addConstrs((p_ij.sum('*',j) == 1 for j in pools), name="Pool_inflow")
# 5.1 Target (min) tolerances
q_pooling.addConstrs((quicksum(content[i][attr]*ik[i,k] for i in sources if (i,k) in s2t)+ quicksum(content[i][attr]*p_ij[i,j]*jk[j,k] for (i,j) in s2p if (j,k) in p2t) >= min_tol[k][attr]*(ik.sum('*',k)+jk.sum('*',k)) for k in targets for attr in min_tol[k].keys()), name="Target_min_tolerances")
# 5.2 Target (max) tolerances
q_pooling.addConstrs((quicksum(content[i][attr]*ik[i,k] for i in sources if (i,k) in s2t)+ quicksum(content[i][attr]*p_ij[i,j]*jk[j,k] for (i,j) in s2p if (j,k) in p2t) <= max_tol[k][attr]*(ik.sum('*',k)+jk.sum('*',k)) for k in targets for attr in max_tol[k].keys()), name="Target_max_tolerances")

# Deploy Objective Function

# 0. Total profit
obj = quicksum(price[k]*(ik.sum('*',k)+jk.sum('*',k)) for k in targets) - quicksum(cost[i]*(quicksum(p_ij[i,j]*jk.sum(j,'*') for j in pools if (i,j) in s2p)+ik.sum(i,'*')) for i in sources)
q_pooling.setObjective(obj, GRB.MAXIMIZE)

# Find the optimal solution
q_pooling.optimize()

Changed value of parameter nonConvex to 2
   Prev: -1  Min: -1  Max: 2  Default: -1
Parameter timelimit unchanged
   Value: 300.0  Min: 0.0  Max: inf  Default: inf
Gurobi Optimizer version 9.0.0 build v9.0.0beta1 (mac64)
Optimize a model with 7 rows, 16 columns and 22 nonzeros
Model fingerprint: 0x082d8c3a
Model has 18 quadratic objective terms
Model has 15 quadratic constraints
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  QMatrix range    [6e-01, 1e+02]
  QLMatrix range   [1e-02, 1e+02]
  Objective range  [9e+01, 2e+02]
  QObjective range [1e+02, 6e+02]
  Bounds range     [1e+00, 8e+02]
  RHS range        [1e+00, 2e+03]
  QRHS range       [5e+02, 2e+04]

Continuous model is non-convex -- solving as a MIP.

Presolve time: 0.00s
Presolved: 95 rows, 35 columns, 315 nonzeros
Presolved model has 18 bilinear constraint(s)
Variable types: 35 continuous, 0 integer (0 binary)

Root relaxation: objective 1.810743e+06, 39 iterations, 0.00 seconds

    Nodes    |    Current Node   

The Q-formulation for this instance has:

- 16 decision variables.
- 7 linear constraints.
- 15 bilinear constraints.
- a bilinear objective function.

Notice it has fewer decision variables and also fewer bilinear constraints. Now Gurobi was able to find the optimal solution of 439,182.59 USD in less than one second, that is five orders of magnitude faster than before.

---
## Analysis

Let's see the optimal flows found using the Q-formulation:

### Flow from Sources to Targets

In [4]:
rows = sources.copy()
columns = targets.copy()
s2t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for source, target in ik.keys():
    if (abs(ik[source, target].x) > 1e-6):
        s2t_plan.loc[source, target] = np.round(ik[source, target].x, 1)
s2t_plan

,t1,t2,t3
s1,0.0,0.0,0.0
s2,966.7,0.0,200.0
s3,0.0,0.0,0.0


### Flow from Pools to Targets

In [5]:
rows = pools.copy()
columns = targets.copy()
p2t_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for pool, target in jk.keys():
    if (abs(jk[pool, target].x) > 1e-6):
        p2t_plan.loc[pool, target] = np.round(jk[pool, target].x, 1)
p2t_plan

,t1,t2,t3
p1,92.8,990.6,0.0
p2,1450.0,0.0,300.0


### Flow from Sources to Pools

In [6]:
rows = sources.copy()
columns = pools.copy()
s2p_plan = pd.DataFrame(columns=columns, index=rows, data=0.0)

for source, pool in p_ij.keys():
    if (abs(p_ij[source, pool].x) > 1e-6):
        s2p_plan.loc[source, pool] = np.round(p_ij[source, pool].x, 1) * p2t_plan.loc[pool,:].sum()
s2p_plan

,p1,p2
s1,325.02,1750.0
s2,216.68,0.0
s3,541.70,0.0


---
## Conclusion

This notebook showed how easy it is to solve Bilinear Programs using Gurobi. It also highlighted the dramatic difference in performance of alternative formulations when solving challenging problems, such as the Standard Pooling Problem. It is thus of utmost importance to analyze carefully the context of the problem at hand, and to weigh the pros and cons of alternative models. 

---
<a id='references'></a>
## References

1. Alfaki, M. (2012). Models and solution methods for the pooling problem.
2. Audet, C., Brimberg, J., Hansen, P., Digabel, S. L., & Mladenović, N. (2004). Pooling problem: Alternate formulations and solution methods. Management science, 50(6), 761-776.
3. Dombrowski, J. (2015, June 07). McCormick envelops. Retrieved from https://optimization.mccormick.northwestern.edu/index.php/McCormick_envelopes
4. Gupte, A., Ahmed, S., Dey, S. S., & Cheon, M. S. (2017). Relaxations and discretizations for the pooling problem. Journal of Global Optimization, 67(3), 631-669.
5. Haverly, C. A. (1978). Studies of the behavior of recursion for the pooling problem. Acm sigmap bulletin, (25), 19-28.
6. Liberti, L. (2008). Introduction to global optimization. Ecole Polytechnique.
7. Zhuang E. (2015, June 06). Spatial branch and bound method. Retrieved from
https://optimization.mccormick.northwestern.edu/index.php/Spatial_branch_and_bound_method

Copyright © 2019 Gurobi Optimization, LLC